# Importamos librerias

In [3]:
import pandas as pd
import numpy as np
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [4]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

    print('Correct Password - connected to SNOWFLAKE')

Connected


# Stock

In [5]:
articulos = tuple(['244746000', '235219000', '230130000', '254840000', '258567000', '245981000', '260668000', '258556000', '260669000', '252289000', '240103000', '250879000', '231713000', '141496000', '250204000', '250609000', '236405000', '252142000', '330482000'])
articulos

('244746000',
 '235219000',
 '230130000',
 '254840000',
 '258567000',
 '245981000',
 '260668000',
 '258556000',
 '260669000',
 '252289000',
 '240103000',
 '250879000',
 '231713000',
 '141496000',
 '250204000',
 '250609000',
 '236405000',
 '252142000',
 '330482000')

In [6]:
query = '''
    SELECT
        --LGL.GEOG_LOCL_COD,
        --LGL.GEOG_LOCL_DESC,
        LAA.ARTC_ARTC_COD,
        LAA.ORIN,
        LAA.ARTC_ARTC_DESC,
        SUM(FS.STCK_UNIDADES) AS UNIDADES
    FROM
        MSTRDB.DWH.FT_STOCK AS FS
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FS.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FS.GEOG_LOCL_ID
    WHERE
        FS.TIEM_DIA_ID = CURRENT_DATE - 1
        AND LAA.ARTC_ARTC_COD IN {articulos_snow}
    GROUP BY
        ALL
'''

cursor.execute(query.format(articulos_snow = articulos))
df = cursor.fetch_pandas_all()
df.head(2)

,ARTC_ARTC_COD,ORIN,ARTC_ARTC_DESC,UNIDADES
0,252142000,1000051653,CHOCLO EN GRANO ODERICH LATA 300 G 1,1030.0
1,258567000,1000054084,SALSA DE TOMATE OLIVE BARILLA 400.00 G 1,1261.0


# En Promo

In [7]:
query = '''
SELECT
    DISTINCT
    EVENTO_DESC,
    ORIN
FROM
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        LAA.ORIN,
        LAA.ARTC_ARTC_COD,
        LAA.ARTC_ARTC_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
    WHERE
        YEAR(FP.PROM_FECHA_FIN) = 2024
        AND FP.PROM_FECHA_INICIO > CURRENT_DATE
        --AND EVENTO_DESC ILIKE '%OFERTATA%'
        AND LAA.ARTC_ARTC_COD IN {articulos_snow}
    )
'''

cursor.execute(query.format(articulos_snow = articulos))
ofertas = cursor.fetch_pandas_all()
ofertas.head(2)

,EVENTO_DESC,ORIN
0,Carnaval2024 - 09/02_18/02,1000054863
1,OfertataFebrero052024-01/02_07/02,1000049220


In [8]:
ofertas_2 = ofertas.groupby('ORIN')['EVENTO_DESC'].sum().reset_index()
ofertas_2

,ORIN,EVENTO_DESC
0,1000047654,OfertataEnero042024-25/01_31/01OfertataFebrero...
1,1000048181,Especial Cervezas 2024 - 08/02_14/02
2,1000049220,OfertataFebrero052024-01/02_07/02Especial Cerv...
3,1000050785,OfertataFebrero062024-08/02_14/02
4,1000054862,OfertataFebrero062024-08/02_14/02
5,1000054863,Carnaval2024 - 09/02_18/02OfertataFebrero06202...
6,1000221905,OfertataEnero042024-25/01_31/01Carnaval2024 - ...


# Venta Proyectada

In [9]:
query = '''
SELECT
    LAA.ORIN,
    SUM(FP.PRONOSTICO_VENTA) AS PRONOSTICO_VENTA
FROM
    MSTRDB.DWH.FT_PROMOS AS FP
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FP.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
WHERE
    YEAR(FP.PROM_FECHA_INICIO) = 2024
        AND LAA.ARTC_ARTC_COD IN {articulos_snow}
GROUP BY
    ALL
'''

cursor.execute(query.format(articulos_snow = articulos))
venta_proyectada = cursor.fetch_pandas_all()
venta_proyectada.head(2)

,ORIN,PRONOSTICO_VENTA
0,1000049626,11778
1,1000051732,14532


# Consolido

In [10]:
consolidado = df.merge(ofertas_2, on = 'ORIN', how = 'left')
consolidado = consolidado.merge(venta_proyectada, on = 'ORIN', how = 'left')
consolidado

,ARTC_ARTC_COD,ORIN,ARTC_ARTC_DESC,UNIDADES,EVENTO_DESC,PRONOSTICO_VENTA
0,252142000,1000051653,CHOCLO EN GRANO ODERICH LATA 300 G 1,1030.0,NaN,3408
1,258567000,1000054084,SALSA DE TOMATE OLIVE BARILLA 400.00 G 1,1261.0,NaN,5928
2,260669000,1000054863,ALFAJOR CHOCOLATE PORTEZUELO PACK 6.00 U.,25634.0,Carnaval2024 - 09/02_18/02OfertataFebrero06202...,21775
3,235219000,1000047733,LECHE DESLACTOSADA PARMALAT 1 L NATURAL,4832.0,NaN,18440
4,252289000,1000051732,PULPA DE TOMATES TAMIZADA COLOLO 1020.00 G 1,4526.0,NaN,14532
5,260668000,1000054862,ALFAJOR NIEVE PORTEZUELO PACK X 6 U,16742.0,OfertataFebrero062024-08/02_14/02,14100
6,236405000,1000047897,HUEVO COLOR YEMADONA X 15.00 U. NATURAL,4556.0,NaN,6736
7,250204000,1000050274,VINAGRE ALCOHOL GAMBERO PET 500.00 CL. 1,2846.0,NaN,5256
8,330482000,1000170088,HAMBURGUESAS SARUBBI X2. 13213,8349.0,NaN,3425
9,240103000,1000048181,CERVEZA HEINEKEN 1 LITRO NO RET.,11211.0,Especial Cervezas 2024 - 08/02_14/02,5808


In [11]:
consolidado.to_excel('Articulos en Promo.xlsx', index = False)

# Fin